In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [50]:
using GLMakie
using Colors
using GeometryBasics
using ReferenceFrameRotations
using FileIO
using CSV, DataFrames
using NativeFileDialog
import YAML

include("gui_utilities.jl")
include("yaml_utilities.jl")

GLMakie.activate!(inline=false)

Quaternion = ReferenceFrameRotations.Quaternion

ReferenceFrameRotations.Quaternion

In [3]:
# Pkg.add("YAML")

### Load Mesh

In [48]:
crazyflie_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"));

## Read CSV

In [80]:
# reading the csv file
df = CSV.read("logs/log1.csv", DataFrame);

# read settings file 
plot_yaml = YAML.load_file("plot.yml"; dicttype=Dict{Symbol,Any})
sim_yaml = YAML.load_file("sim.yml"; dicttype=Dict{Symbol,Any})

plot_params = recursive_dict_to_namedtuple(plot_yaml);
sim_params = recursive_dict_to_namedtuple(sim_yaml);

Quaternion{Float64}:
  + 1.0 + 0.0⋅i + 0.0⋅j + 0.0⋅k

In [65]:
q = Quaternion{Float64}(I)  

Quaternion{Float64}:
  + 1.0 + 0.0⋅i + 0.0⋅j + 0.0⋅k

### Create figure 

In [68]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = get_primary_resolution(1))

display(f)

GLMakie.Screen(...)

In [78]:
include("gui_utilities.jl")
include("yaml_utilities.jl")

# read settings file 
params_yaml = YAML.load_file("plot.yml"; dicttype=Dict{Symbol,Any})
plot_params = recursive_dict_to_namedtuple(params_yaml );

# clear figure 
empty!(f)

# Params --------------------------------------------

vis_params = plot_params.visualizer
graph_params = plot_params.graph

configs_vec = get_configs_vec(plot_params.graph.axis.configs)


# Observables -------------------------------------------------
sim_time_obs = Observable{Float64}(0.0)
sim_state_obs = Observable{Bool}(false)

# Top level
g_top = f[0, 1:2] = GridLayout()
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout()

# Add title
supertitle1 = Label(g_top[1, 1:2], plot_params.title.name, fontsize = 60)
time_title = Label(g_top[2, 1:2], "Time = "* string(sim_time_obs[]) *" s", fontsize = 40)

# Box(f[0, 1:2], color = (:red, 0.2), strokewidth = 0)

# Right Grid 
g_right_plots = g_right[1, 1]= GridLayout()
g_state_plots = g_right_plots[1, 1]= GridLayout()
g_control_plots = g_right_plots[2, 1]= GridLayout()

g_right_widgets = g_right[2, 1]=  GridLayout()

# Left Grid 
g_left_plots = g_left[1, 1]= GridLayout()
g_left_widgets = g_left[2, 1]=  GridLayout(tellwidth = false)


# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# 3d axis for airplane visualization
vis_ax = Axis3(g_left_plots[1, 1],
    title = vis_params.title,
    limits =  (vis_params.axis.low,vis_params.axis.high, vis_params.axis.low,vis_params.axis.high, vis_params.axis.low,vis_params.axis.high),
    aspect = (vis_params.axis.aspect_x, vis_params.axis.aspect_y, vis_params.axis.aspect_z),
    xlabel= vis_params.axis.labels.x, xlabelsize=vis_params.axis.label_size,
    ylabel= vis_params.axis.labels.y, ylabelsize=vis_params.axis.label_size,
    zlabel= vis_params.axis.labels.z, zlabelsize=vis_params.axis.label_size,
    halign =:left,
    xspinecolor_1 = :white,
    xspinecolor_3 = :white,
    yspinecolor_1 = :white,
    yspinecolor_3 = :white,
    zspinecolor_1 = :white,
    zspinecolor_3 = :white,
    xspinewidth= vis_params.axis.spine_width,
    yspinewidth= vis_params.axis.spine_width,
    zspinewidth= vis_params.axis.spine_width,
    xlabeloffset= vis_params.axis.label_offset,
    ylabeloffset= vis_params.axis.label_offset,
    zlabeloffset= vis_params.axis.label_offset,
    xgridwidth = vis_params.axis.grid_width,
    ygridwidth = vis_params.axis.grid_width,
    zgridwidth = vis_params.axis.grid_width,
)

# force 3d visualizer to have an aspect ratio of 1
rowsize!(g_left, 1, Aspect(1, 1.0))

m = mesh!(vis_ax, crazyflie_stl , color=:red)

scale!(m, vis_params.mesh.scale, vis_params.mesh.scale, vis_params.mesh.scale)

# center mesh at the origin
translate!(m, Vec3f(vis_params.mesh.initial_translation[1], vis_params.mesh.initial_translation[2], vis_params.mesh.initial_translation[3]))

# apply initial orientation
rotate_mesh(to_quaternion(sim_params.vehicle.initial_state.q_BI))

state_plots = Axis[]
control_plots = Axis[]

for i in 1:graph_params.n_state
    plot = Axis(f, ylabel=graph_params.ylabels[i],titlesize=graph_params.titlesize )
    push!(state_plots, plot)

    # g_right_plots[i,1] = state_plots[i]
    g_state_plots[i,1] = state_plots[i]
end

for i in 1:graph_params.n_control
    plot = Axis(f,
    # ylabel=graph_params.ylabels[i] 
    )
    push!(control_plots, plot)

    # g_right_plots[i,1] = state_plots[i]
    g_control_plots[i,1] = control_plots[i]
end

# slider grid for timeline control
timeline_slider = Slider(f, range = 0:0.01:10, startvalue = 0 , linewidth = 25.0, tellheight = false,
                     halign =:left)

#timeline button
timeline_btn = Button(f, label = "Play", tellwidth=false, halign =:center, fontsize=40)
timeline_left_label = Label(f,"0.0 s", justification = :left)
timeline_right_label = Label(f,"10.0 s", justification = :left)

g_left_widgets[1,1] = timeline_left_label
g_left_widgets[1,2] = timeline_slider
g_left_widgets[1,3] = timeline_right_label

g_left_widgets[2,:] = timeline_btn

    
# how much to shrink control plots grid
rowsize!(g_right_plots, 2,  Auto(0.2))

# shrink right widgets grid to make space for plots
# rowsize!(g_right, 2,  Auto(0.2))

# attitude reset button
attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)
g_right_widgets[1,1] = attitude_reset_btn

# dropdown menu
config_menu = Menu(f,
    options = configs_vec,
    default = "yzθ")

g_right_widgets[1,2] = config_menu

# toggle buttons
toggles = [Toggle(f, active = active) for active in [true, true, true]]
labels = [Label(f, label) for label in ["y", "z", "θ"]]

g_right_toggles = g_right_widgets[1,3] =  GridLayout()

g_right_toggles[1,1] = grid!(hcat(toggles[1], labels[1]), tellheight = false, tellwidth = false)
g_right_toggles[1,2] = grid!(hcat(toggles[2], labels[2]), tellheight = false, tellwidth = false)
g_right_toggles[1,3] = grid!(hcat(toggles[3], labels[3]), tellheight = false, tellwidth = false)

g_right[2, 1] = g_right_widgets


# event handling
on(config_menu.selection) do config
    plot_trajectory(configs_vec, config)
end

on(timeline_btn.clicks) do clicks

    # if sim is not already running, start sim
    if sim_state_obs[] ==false
        
        # set sim state flag to true
        sim_state_obs[] = true

        # change button text to show "Stop"
        timeline_btn.label = "Stop"

        # start sim task asynchronously
        sim_task =  @async plot_3d_trajectory(duration=3.0, sim_time_obs=sim_time_obs, sim_state_obs=sim_state_obs)
        #plot_3d_trajectory(duration=3.0, sim_time_obs=sim_time_obs, sim_state_obs=sim_state_obs)
        
        @async begin 
            # wait for 3d trajectory plotting to finish
             wait(sim_task)
            
            # set sim state flag to false 
            sim_state_obs[] = false
    
            # change button text to show "Play"
            timeline_btn.label = "Play"
        end

    else
        # if sim is currently running, set sim state flag to false
        sim_state_obs[] = false
    end

end


# lift(timeline_slider.value) do val
#     # sim_time_obs[] = val
# end

on(sim_time_obs) do time
    time_title.text = "Time: " * string(time) * " s"
end

time_marker = lift(sim_time_obs) do time_val
    time_val
end

# initial setup
# plot_trajectory(config_dict, config_menu.selection[])
plot_trajectory(configs_vec, config_menu.selection[])

traj_count_n = 1000
lines!(vis_ax, zeros(traj_count_n), df.y_req[1:traj_count_n], df.z_req[1:traj_count_n], linestyle=:dash)

trim!(f.layout)

# display(f)

ReferenceFrameRotations.Quaternion

# Second window 

In [10]:
fig = Figure(resolution=(1080, 720))
g_right = fig[1, 1] = GridLayout()

menu1 = Menu(fig, options = ["Cascaded Controller", "Monolithic", "C"], tellwidth=false)
menu2 = Menu(fig, options = ["PID", "LQR", "C"], tellwidth=false)

g_right[1, 1] = hgrid!(
    Label(fig, "Controller Architecture: "), menu1)

g_right[2, 1] = hgrid!(
    Label(fig, "Controller : "), menu2)


button1 = Button(fig[2, 1], label = "Open file", tellwidth=false)
button2 = Button(fig[3, 1], label = "Open folder", tellwidth=false)

on(button1.clicks) do n
    @async begin
        f = pick_file()
        @show f
    end
    
end

on(button2.clicks) do n
    @async begin
        f = pick_folder()
        @show f
    end
end


# ---------------------------------------------
# screen properties   
flag =  @isdefined screen2
if flag==true
    GLMakie.destroy!(screen2)
end

screen2 = GLMakie.Screen()
display(screen2, fig)
GLMakie.set_screen_visibility!(screen2, true)

In [42]:
f = Figure(resolution = (800, 800))

for i in 1:3, j in 1:3
    Axis(f[i, j])
end

Label(f[0, :], text = "First Supertitle", fontsize = 20)
Label(f[-1, :], text = "Second Supertitle", fontsize = 30)
Label(f[-2, :], text = "Third Supertitle", fontsize = 40)

display(f)

GLMakie.Screen(...)